In [ ]:
import os
import scipy.io
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torchvision.models as models
from adabound import AdaBound  
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

class CarsDataset(Dataset):
    def __init__(self, annos, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform

        self.filenames = []
        self.labels = []

        for i in range(annos['annotations'].shape[1]):
            annotation = annos['annotations'][0, i]
            fname = annotation['fname'][0]
            class_label = int(annotation['class'][0][0])

            
            if isinstance(fname, bytes):
                fname = fname.decode('utf-8')
            elif isinstance(fname, np.ndarray):
                
                fname = ''.join([chr(c) for c in fname])

            
            if not fname.endswith('.jpg'):
                fname += '.jpg'

            img_path = os.path.join(img_dir, fname)
            if os.path.exists(img_path):
                self.filenames.append(fname)
                self.labels.append(class_label)
            else:
                print(f"Файл отсутствует: {img_path}")

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.filenames[idx])
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Ошибка при загрузке изображения {img_path}: {e}")
            raise
        label = self.labels[idx] - 1  
        if self.transform:
            image = self.transform(image)
        return image, label

class VGG16Custom(nn.Module):
    def __init__(self, num_classes):
        super(VGG16Custom, self).__init__()
        self.model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        
        for param in self.model.features.parameters():
            param.requires_grad = False
        
        self.model.classifier[6] = nn.Linear(4096, num_classes)

    def forward(self, x):
        return self.model(x)

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Оценка на тестовом наборе"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_test_loss = running_loss / len(test_loader.dataset)
    epoch_acc = 100 * correct / total
    return epoch_test_loss, epoch_acc

def train_model(model, optimizer, train_loader, criterion, device, num_epochs=5):
    train_losses = []
    train_accuracies = []
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nНачало эпохи {epoch + 1}/{num_epochs}")
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Эпоха {epoch+1}/{num_epochs}"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
        print(f"Эпоха {epoch+1}/{num_epochs} | Потери на обучении: {epoch_loss:.4f} | Точность на обучении: {epoch_accuracy:.2f}%")

    total_time = time.time() - start_time
    print(f"Обучение завершено за {total_time/60:.2f} минут.")
    return train_losses, train_accuracies

if __name__ == "__main__":
    
    train_dir = 'C:\\Users\\latip\\OneDrive\\Рабочий стол\\Учеба\\4КС1\\ИИ\\Лаба1\\cars_train\\cars_train'
    test_dir = 'C:\\Users\\latip\\OneDrive\\Рабочий стол\\Учеба\\4КС1\\ИИ\\Лаба1\\cars_test\\cars_test'

    
    train_annos = scipy.io.loadmat('C:\\Users\\latip\\OneDrive\\Рабочий стол\\Учеба\\4КС1\\ИИ\\Лаба1\\ДатаСет\\cars_train_annos.mat')
    test_annos = scipy.io.loadmat('C:\\Users\\latip\\OneDrive\\Рабочий стол\\Учеба\\4КС1\\ИИ\\Лаба1\\ДатаСет\\cars_test_annos_withlabels_eval.mat')
    meta = scipy.io.loadmat('C:\\Users\\latip\\OneDrive\\Рабочий стол\\Учеба\\4КС1\\ИИ\\Лаба1\\ДатаСет\\cars_meta.mat')

    
    class_names = [name[0] for name in meta['class_names'][0]]
    num_classes = len(class_names)


    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    
    train_dataset = CarsDataset(train_annos, train_dir, transform=transform)
    test_dataset = CarsDataset(test_annos, test_dir, transform=transform)

    train_subset = torch.utils.data.Subset(train_dataset, list(range(500)))
    test_subset = torch.utils.data.Subset(test_dataset, list(range(500)))

    print(f"Количество тренировочных данных: {len(train_dataset)}")
    print(f"Количество тестовых данных: {len(test_dataset)}")

    
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=0, pin_memory=torch.cuda.is_available())
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

    
    model = VGG16Custom(num_classes=num_classes)

   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Используемое устройство: {device}")

    
    learning_rate = 1e-3
    weight_decay = 5e-4

    optimizer_adabound = AdaBound(model.parameters(), lr=learning_rate, final_lr=0.1, weight_decay=weight_decay)
    optimizer_adam = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    
    criterion = nn.CrossEntropyLoss()

    
    num_epochs = 5 
    print("\nОбучение с оптимизатором AdaBound:")
    train_losses_adabound, train_accuracies_adabound = train_model(
        model, optimizer_adabound, train_loader, criterion, device, num_epochs=num_epochs
    )

    
    print("\nОценка модели после обучения с AdaBound:")
    test_loss_adabound, test_acc_adabound = evaluate_model(model, test_loader, criterion, device)
    print(f"Тестовая точность AdaBound: {test_acc_adabound:.2f}%")

    
    model = VGG16Custom(num_classes).to(device)

    
    optimizer_adam = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    
    print("\nОбучение с оптимизатором Adam:")
    train_losses_adam, train_accuracies_adam = train_model(
        model, optimizer_adam, train_loader, criterion, device, num_epochs=num_epochs
    )

    
    print("\nОценка модели после обучения с Adam:")
    test_loss_adam, test_acc_adam = evaluate_model(model, test_loader, criterion, device)
    print(f"Тестовая точность Adam: {test_acc_adam:.2f}%")

    
    optimizers = ['AdaBound', 'Adam']
    test_accuracies = [test_acc_adabound, test_acc_adam]

    plt.figure(figsize=(6, 6))
    bars = plt.bar(optimizers, test_accuracies, color=['blue', 'orange'])
    plt.xlabel('Оптимизатор')
    plt.ylabel('Тестовая точность (%)')
    plt.title('Сравнение тестовой точности оптимизаторов')
    plt.ylim(0, 100)

    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 1, f'{height:.2f}%', ha='center', va='bottom')

    plt.show()
